ИМПОРТ БИБЛИОТЕК

In [ ]:
import os
import pandas as pd
from nltk import word_tokenize

import nltk
nltk.download('punkt') 

from nltk.tokenize import WordPunctTokenizer

nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words('russian')

import string

!pip install pymorphy2
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np

from sklearn.decomposition import LatentDirichletAllocation

!pip install pyldavis

import pyLDAvis
import pyLDAvis.sklearn

ЗАГРУЗКА ДАННЫХ

In [49]:
def get_data():     
    corpus = []
    for filename in os.listdir():
        if filename.endswith(".txt"):
            with open (filename, 'r', encoding = 'utf-8') as f:
                text = f.read()
                corpus.append(text)
    return corpus


In [ ]:
corpus = get_data()
corpus

СОЗДАНИЕ ТАБЛИЦЫ С ДАННЫМИ

In [51]:
df = pd.DataFrame(corpus, columns = ['text']) 

In [52]:
df.head()

,text
0,Российский научный фонд подвел итоги двух реги...
1,Ученые Томского политехнического университета ...
2,"Два проекта, реализуемые в лабораториях Фабла..."
3,"Даниил Снетков, первокурсник кафедры механики ..."
4,Ученые ТПУ разрабатывают универсальный комплек...


ТОКЕНИЗАЦИЯ И ЛЕММАТИЗАЦИЯ ТЕКСТА

In [53]:
def preprocess_data(text): 
    text = text.lower()
    text_tokens = WordPunctTokenizer().tokenize(text)
    spec_chars = string.punctuation + '\n\xa0«»\t—...'
    text_tokens = [token for token in text_tokens if 
                   (token not in stopwords and token not in spec_chars and not any(char.isdigit() for char in token))]

    text_lemmatized = [morph.parse(token)[0].normal_form for token in text_tokens]

    return " ".join(text_lemmatized)
    

In [54]:
df = df.applymap(preprocess_data) # загружаем обработанный текст в нашу таблицу

In [55]:
preprocess_data(df.iloc[0].text)

'российский научный фонд подвести итог региональный конкурс проект научный группа малый научный группа результат поддержка получить шесть проект учёный томский политехнический университет это хороший результат среди университет томск конкурс участвовать тысяча заявка исследователь регион россия результат экспертиза поддержать проект научный группа проект малый научный группа общий сумма паритетный поддержка сторона фонд регион составить миллиард рубль итог конкурс год получение грант рнф приоритетный направление деятельность рнф проведение фундаментальный научный исследование поисковый научный исследование отдельный научный группа региональный конкурс поддержка получить шесть проект томский область реализоваться учёный томский политех электронный компонент основа подход лазерный интеграция биосовместимый биоразлагать гибкий электронный схема руководство профессор исследовательский школа химический биомедицинский технология тпу евгений шеремета разработка биологически активный наноматер

In [56]:
df.head()

,text
0,российский научный фонд подвести итог два реги...
1,учёный томский политехнический университет ско...
2,проект реализовать лаборатория фаблабнуть поли...
3,даниил снетковый первокурсник кафедра механика...
4,учёный тпу разрабатывать универсальный комплек...


ЧАСТОТНЫЙ АНАЛИЗ

In [57]:
type(df['text'].tolist()) # делаем список
texts = df['text'].tolist()

ВЕКТОРИЗАЦИЯ

In [58]:
vectorizer = CountVectorizer(ngram_range=(1, 2))

In [59]:
X = vectorizer.fit_transform(texts)

In [60]:
X.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

Tf-Idf

In [61]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2))

In [ ]:
tfidf = tfidf_vectorizer.fit_transform(texts)   
print(tfidf)  

In [ ]:
tfidf.shape
tfidf.todense()[0].tolist() 

In [ ]:
tfidf_vectorizer.vocabulary_

In [ ]:
words = tfidf_vectorizer.get_feature_names()[:10]

In [66]:
np.argsort(tfidf_vectorizer.vocabulary_)

array([0])

In [67]:
def get_top_tf_idf_words(tfidf_vector, feature_names, top_n):
    sorted_nzs = np.argsort(tfidf_vector.data)[:-(top_n+1):-1]
    return feature_names[tfidf_vector.indices[sorted_nzs]]

In [68]:
df.text.iloc[0]

'российский научный фонд подвести итог два региональный конкурс проект научный группа малый научный группа результат поддержка получить шесть проект учёный томский политехнический университет это хороший результат среди университет томск конкурс участвовать тысяча заявка исследователь регион россия результат экспертиза поддержать проект научный группа проект малый научный группа общий сумма паритетный поддержка сторона фонд регион составить миллиард рубль итог конкурс год получение грант рнф приоритетный направление деятельность рнф проведение фундаментальный научный исследование поисковый научный исследование отдельный научный группа региональный конкурс поддержка получить шесть проект томский область реализоваться учёный томский политех электронный компонент основа подход лазерный интеграция биосовместимый биоразлагать гибкий электронный схема руководство профессор исследовательский школа химический биомедицинский технология тпу евгения шеремета разработка биологически активный наном

In [69]:
feature_names = np.array(tfidf_vectorizer.get_feature_names())

for i, article in enumerate(df.text.head()):
        article_vector = tfidf[i, :]
        words = get_top_tf_idf_words(article_vector, feature_names, 10)
        print(article)
        print(words, '\n')

российский научный фонд подвести итог два региональный конкурс проект научный группа малый научный группа результат поддержка получить шесть проект учёный томский политехнический университет это хороший результат среди университет томск конкурс участвовать тысяча заявка исследователь регион россия результат экспертиза поддержать проект научный группа проект малый научный группа общий сумма паритетный поддержка сторона фонд регион составить миллиард рубль итог конкурс год получение грант рнф приоритетный направление деятельность рнф проведение фундаментальный научный исследование поисковый научный исследование отдельный научный группа региональный конкурс поддержка получить шесть проект томский область реализоваться учёный томский политех электронный компонент основа подход лазерный интеграция биосовместимый биоразлагать гибкий электронный схема руководство профессор исследовательский школа химический биомедицинский технология тпу евгения шеремета разработка биологически активный нанома

In [70]:
df_2 = df.applymap

ТЕМАТИЧЕСКОЕ МОДЕЛИРОВАНИЕ

LDA

In [71]:
lda = LatentDirichletAllocation(n_components=10, random_state=0) # выбираем 10 самых популярных тем

In [72]:
lda.fit(tfidf)

LatentDirichletAllocation(random_state=0)

In [73]:
pyLDAvis.enable_notebook()

In [74]:
pyLDAvis.sklearn.prepare(lda,tfidf,tfidf_vectorizer)

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.029241  0.000238       1        1  36.623475
1      0.005249  0.004839       2        1  12.366987
6      0.004111 -0.001371       3        1   9.368909
9      0.003587 -0.001228       4        1   7.892363
7      0.003355 -0.000970       5        1   7.571351
8      0.003234 -0.000446       6        1   7.005358
0      0.003156 -0.000586       7        1   6.737418
4      0.002912 -0.000899       8        1   5.474096
5      0.001830  0.000225       9        1   3.563365
2      0.001808  0.000198      10        1   3.396676, topic_info=              Term      Freq     Total Category  logprob  loglift
38387  университет  0.000000  0.000000  Default  30.0000  30.0000
5867           год  0.000000  0.000000  Default  29.0000  29.0000
36816   технология  0.000000  0.000000  Default  28.0000  28.0000
27371       проект  0.000000  0.000000  Default  27.0000  27.0000
13696      который  0.000000  0.000000  Default  26.0000  26.0000
...            ...       ...       ...      ...      ...      ...
22462  переработка  0.006629  0.083415  Topic10  -9.6224   0.8500
30002      реактор  0.006673  0.096250  Topic10  -9.6157   0.7135
19742     общество  0.005898  0.080069  Topic10  -9.7392   0.7741
23034       плёнка  0.005818  0.081175  Topic10  -9.7528   0.7467
16517  модификация  0.006072  0.091557  Topic10  -9.7101   0.6691

[411 rows x 6 columns], token_table=       Topic      Freq          Term
term                                
4643       1  1.782734           вуз
5867       1  1.391092           год
11353      1  1.573965  исследование
13696      1  1.440139       который
17859      1  1.508846       научный
18750      1  1.794742         новый
24171      1  1.781168       политех
27062      1  1.509504     программа
27371      1  1.403761        проект
28551      1  1.834102        работа
28975      1  1.844531      развитие
29376      1  1.756960    разработка
30419      1  1.620336       рейтинг
34137      1  1.507812          спбп
35501      1  1.515331       студент
36816      1  1.389501    технология
38387      1  1.149309   университет
41456      1  1.730295         школа
42376      1  1.769964           это, R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 7, 10, 8, 9, 1, 5, 6, 3])